In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
train  = pd.read_csv("data/train.csv")

# Data Viz

In [ ]:
# from dataprep.eda import create_report

# create_report(train).show()

In [ ]:
train.columns

In [ ]:
# from dataprep.eda import plot, plot_correlation, plot_missing

# plot(train, "Pclass", "Survived")

In [ ]:
# plot(train,"Survived", "Pclass")

In [ ]:
# plot(train,"Survived", "Pclass")

# Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

X = train[[col for col in train.columns if col != "Survived"]]
y = train["Survived"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y,  test_size=0.3, random_state=42, shuffle=True, stratify=y)

In [ ]:
train.head()

# Preprocessing

In [ ]:
from john_toolbox.preprocessing.pandas_pipeline import PandasPipeline
from src.preprocessing import (
    conformity_column_list,
    data_cleaning_list,
    encoder_list
)

## Define PandasPipeline

In [ ]:
step_list = conformity_column_list + data_cleaning_list + encoder_list

pipeline = PandasPipeline(
    steps=step_list, target_name="Survived", verbose=True)

In [ ]:
train.Cabin

## Fit transform

In [ ]:
train_transformed = pipeline.fit_transform(
    df=pd.concat([X_train, y_train], axis=1))

In [ ]:
X_train = train_transformed[[col for col in train_transformed.columns if col != "Survived"]]
y_train = train_transformed[["Survived"]]

In [ ]:
X_train

# TRAIN with XGB

In [ ]:
from xgboost import XGBClassifier

params = {
    "eta": 0.01, #  0.01-0.2
}
xgb = XGBClassifier()

In [ ]:
xgb.fit(X_train, y_train)

# Evaluation

In [ ]:
from john_toolbox.evaluation.visualisation import plot_curves, plot_classification_report

## Train

In [ ]:
y_pred_train_prob = xgb.predict_proba(X_train)[:, 1]
y_pred_train = xgb.predict(X_train)

In [ ]:
plot_classification_report(y_train, y_pred_train, size=(5, 5))

In [ ]:
plot_curves(y_train, y_pred_train)

## Valid

In [ ]:
valid_transformed = pipeline.transform(df=pd.concat([X_valid, y_valid], axis=1))

In [ ]:
X_valid = valid_transformed[[col for col in valid_transformed.columns if col != "Survived"]]

In [ ]:
y_pred_valid_prob = xgb.predict_proba(X_valid)[:, 1]
y_pred_valid = xgb.predict(X_valid)

In [ ]:
plot_classification_report(y_valid, y_pred_valid, size=(5, 5))

In [ ]:
plot_curves(y_valid, y_pred_valid)

In [ ]:
from sklearn.metrics import classification_report

# Prediction

In [ ]:
test  = pd.read_csv("data/test.csv")

In [ ]:
train 

In [ ]:
test_transformed = pipeline.transform(test)
X_test = test_transformed[[col for col in train_transformed.columns if col != "Survived"]]

In [ ]:
y_preds = xgb.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': y_preds})
output.to_csv('my_submission.csv', index=False)